In [5]:
from rl.environments.simple_conveyor_9 import simple_conveyor_9
import yaml
import ray
from ray import tune

config_path = 'rl/config/simple_conveyor_9.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f)
env = simple_conveyor_9(config)

c:\users\_vinc\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
INFO:root:queues that are initialized: [[2, 3, 3, 2, 2, 2, 2, 2, 3, 3], [3, 2, 2, 2, 3, 2, 3, 2, 2, 2], [3, 2, 3, 3, 3, 2, 2, 2, 2, 3]]


In [6]:
ray.init(ignore_reinit_error=True)
config = {
    'env': 'CartPole-v0'
}
stop = {
    'timesteps_total': 10000
}
results = tune.run(
    'PPO', # Specify the algorithm to train
    config=config,
    stop=stop
)

2020-10-01 11:30:59,712	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.


ModuleNotFoundError: No module named 'tree'

(pid=None) F1001 11:35:53.758596 19392  3088 redis_client.cc:74]  Check failed: num_attempts < RayConfig::instance().redis_db_connect_retries() Expected 1 Redis shard addresses, found 0
(pid=None) *** Check failure stack trace: ***
(pid=None)     @   00007FF76569729B  public: void __cdecl google::LogMessage::Flush(void) __ptr64
(pid=None)     @   00007FF7656960F2  public: __cdecl google::LogMessage::~LogMessage(void) __ptr64
(pid=None)     @   00007FF76565CB28  public: virtual __cdecl google::NullStreamFatal::~NullStreamFatal(void) __ptr64
(pid=None)     @   00007FF76553789E  public: class google::LogMessage::LogStream * __ptr64 __cdecl google::LogMessage::LogStream::self(void)const __ptr64
(pid=None)     @   00007FF765536A47  public: class google::LogMessage::LogStream * __ptr64 __cdecl google::LogMessage::LogStream::self(void)const __ptr64
(pid=None)     @   00007FF765536494  public: class google::LogMessage::LogStream * __ptr64 __cdecl google::LogMessage::LogStream::self(void)const 